In [ ]:
import os
import pandas as pd 
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSCanonical, PLSRegression, CCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, cross_val_score
# from sklearn.manifold import TSNE
import matplotlib  as mpl
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
%matplotlib inline
import os, shutil, glob
from random import randint
import seaborn as sns; sns.set_style("white")
import umap as umap

# import plotnine as gg
# from cytominer_eval import evaluate

os.getcwd()

In [ ]:
figformat = 'png'
dpi = 300
statarg = 'single'
OutputDir = f'./output/4_Clusterplots'
if not os.path.exists(OutputDir): 
    os.makedirs(OutputDir)
FeatureDir = './output/1_FeaturesImages'


In [ ]:
read_data = pd.read_parquet(f'{FeatureDir}/spheroidSingleCell.parquet')
cbar = pd.DataFrame()
read_data.reset_index(inplace=True, drop = True)
training_data = read_data.copy()
loc_data = training_data[['Nuclei_Location_Center_X','Nuclei_Location_Center_Y']]
training_data = training_data.loc[:,~training_data.columns.str.contains('Location', case=True)]
training_data = training_data.loc[:,~training_data.columns.str.contains('ImageNumber_', case=True)]
training_data = training_data.loc[:,~training_data.columns.str.contains('Parent', case=True)]
training_data = training_data.loc[:,~training_data.columns.str.contains('Children', case=True)]
training_data = training_data.loc[:,~training_data.columns.str.contains('_ObjectNumber', case=True)]
training_data = training_data.loc[:,~training_data.columns.str.contains('_Object_Number', case=True)]
training_data = training_data.loc[:,~training_data.columns.str.contains('_Y', case=True)]
training_data = training_data.loc[:,~training_data.columns.str.contains('_X', case=True)]
##### Correlation contains some kind of well information
# feature_groups = ['AreaShape'
#                   , 'Correlation'
#                   , 'Granularity'
#                   , 'Intensity'
#                   , 'Neighbors'
#                   , 'RadialDistribution'
#                   ]
# real_names = [x for x in training_data.columns if x.split('_')[0] in feature_groups]
# training_data = training_data[real_names]

### New Onehot
target = pd.DataFrame()
onehot_val = list(range(len(training_data['Metadata_Cmpd'].unique())))
onehot_dict = dict(zip(training_data['Metadata_Cmpd'].unique(), onehot_val))
target['onehot'] = training_data['Metadata_Cmpd'].apply(lambda x: onehot_dict.get(x, -1))
training_data['onehot'] = target['onehot']

training_data = training_data.copy()



In [ ]:
training_data.head()

In [ ]:
toNpy = training_data.loc[:,~training_data.columns.str.contains('Metadata_|onehot')]
toNpy.reset_index(inplace=True, drop=True)
toNpy = toNpy.copy()
dataNpy = toNpy.to_numpy()
colnames = toNpy.columns

In [ ]:
np.isnan(dataNpy).any()


In [ ]:
np.isinf(dataNpy).any()

In [ ]:
dataPCA = pd.DataFrame()
dataPCA['Metadata_Cmpd'] = training_data['Metadata_Cmpd']
dataPCA['onehot'] = training_data['onehot']

dataUMAP = pd.DataFrame()
dataUMAP['Metadata_Cmpd'] = training_data['Metadata_Cmpd']
dataUMAP['onehot'] = training_data['onehot']

In [ ]:
# perplex = 20
pca_model = PCA(n_components=  2)
pca_model = pca_model.fit(dataNpy)
pcaOut = pca_model.transform(dataNpy)
dataPCA['pc1'] = pcaOut[:,0]
dataPCA['pc2'] = pcaOut[:,1]
dataPCA = dataPCA.copy()

In [ ]:
# cmap = sns.color_palette("hls", n_colors=11)
# cmap = sns.color_palette("Set3", n_colors=12)
# cmap = cmap[:11]
cmap="RdBu_r"
# hue = dataPCA['Metadata_Cmpd']
# hue = training_data['cluster']

fig = plt.figure(figsize=[14, 5])
ax = fig.add_subplot(121)
ax.set_xlabel('PC 1', fontsize = 10)
ax.set_ylabel('PC 2', fontsize = 10)
ax.spines['top'].set_color('w')
ax.spines['right'].set_color('w')
ax.spines['left'].set_color('grey')
ax.spines['bottom'].set_color('grey')


sns.set(font_scale=0.9)
sns_plot = sns.clustermap(x, method='ward', metric='euclidean', z_score=None,
                        standard_scale=None,  cbar_kws=None, figsize=(12, nr_rows/4 +1), 
                        row_cluster=False, col_cluster=True,
                        cbar_pos=(0.02, 0.8, 0.05, 0.18), 
                        row_linkage=None, col_linkage=None, row_colors=None,
                                vmin =-5, vmax=5,
                        col_colors=None, cmap="RdBu_r", xticklabels=False )
plt.suptitle('Project {}, clustermap for {}', fontsize = 12, y=1.05)



ax.set_facecolor('w')
ax.get_legend().remove()
